In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def read_output(file_name):
    with open(file_name,"r") as file:
        epochs=[]
        train_perplexity=[]
        val_perplexity=[]
        for line in file:
            x,y,z=line.strip().split(',')
            epochs.append(int(x))
            train_perplexity.append(float(y))
            val_perplexity.append(float(z))
    return epochs, train_perplexity, val_perplexity

In [ ]:
model_name="longformer-base"
output_dir=model_name.split("-")[0] + "_" + model_name.split("-")[1] + "_repo"
file_name=os.path.join(os.getcwd(),output_dir,"Perplexity.txt")
epochs, _, longformer_base_perplexity=read_output(file_name)

In [ ]:
model_name="longformer-large"
output_dir=model_name.split("-")[0] + "_" + model_name.split("-")[1] + "_repo"
file_name=os.path.join(os.getcwd(),output_dir,"Perplexity.txt")
epochs, _, longformer_large_perplexity=read_output(file_name)

In [ ]:
model_name="roberta-base"
output_dir=model_name.split("-")[0] + "_" + model_name.split("-")[1] + "_repo"
file_name=os.path.join(os.getcwd(),output_dir,"Perplexity.txt")
epochs, _, roberta_base_perplexity=read_output(file_name)

In [ ]:
model_name="roberta-large"
output_dir=model_name.split("-")[0] + "_" + model_name.split("-")[1] + "_repo"
file_name=os.path.join(os.getcwd(),output_dir,"Perplexity.txt")
epochs, _, roberta_large_perplexity=read_output(file_name)

In [ ]:
model_name="bigbird-roberta-large"
output_dir=model_name.split("-")[0] + "_" + model_name.split("-")[1] + "_repo"
file_name=os.path.join(os.getcwd(),output_dir,"Perplexity.txt")
epochs, _, bigbird_roberta_perplexity=read_output(file_name)

In [ ]:
fig = plt.figure(figsize=(8, 6))

# Plot each model's performance over time as a line chart
plt.plot(epochs[:len(roberta_base_perplexity)], roberta_base_perplexity, label='roberta-base', color='purple', linewidth=2, linestyle='-', marker="D")
plt.plot(epochs[:len(roberta_large_perplexity)], roberta_large_perplexity, label='roberta-large', color='blue', linewidth=2, linestyle='--', marker="o")
plt.plot(epochs[:len(longformer_base_perplexity)], longformer_base_perplexity, label='longformer-base', color='red', linewidth=2, linestyle=':', marker="s")
plt.plot(epochs[:len(longformer_large_perplexity)], longformer_large_perplexity, label='longformer-large', color='green', linewidth=2, linestyle='-.', marker="^")
# plt.plot(epochs[:len(bigbird_roberta_perplexity)], bigbird_roberta_perplexity, label='BigBird', color='orange', linewidth=2, linestyle='-.', marker=">")

# Set chart title and axis labels
plt.title('Perplexity over Epochs',fontsize=18)
plt.xlabel('Epoch',fontsize=12)
plt.ylabel('Perplexity',fontsize=12)
plt.xticks(epochs)
# Add a legend to the chart
plt.legend()

# Display the chart
plt.show()

In [ ]:
import sys
sys.path.append("/opt/omniai/work/instance1/jupyter/v3_new_email/Fine-Tuning")
sys.path=list(set(sys.path))

path_to_remove=["/opt/omniai/work/instance1/jupyter/v2_new_email/","/opt/omniai/work/instance1/jupyter/v2_new_email/Fine-Tuning"]
for path in path_to_remove:
    if path in sys.path:
        sys.path.remove(path)
        
import utils

In [ ]:
def metrics_read(df, model_name):
    # df=pd.read_csv(os.path.join(output_dir , file_name))
    true_y=df["True_label"].values
    pred_y=df["Predicted_label"].values
    pred_prob=df["Predicted_prob"].values
    best_threshold=df['best_threshold'].unique()[0]

    # test_output=utils.model_evaluate(true_y.reshape(-1),pred_y)
    test_output=utils.model_evaluate(true_y.reshape(-1),pred_prob,best_threshold)
    metric=pd.DataFrame()
    metric["model_type"]=[f"{model_name}"]
    metric["total complaint #"]=[test_output["total positive"]]
    metric["false_positive"]=[test_output["false positive"]]
    metric["false_negative"]=[test_output["false_negative"]]
    metric["precision"]=[test_output["precision"]]
    metric["recall"]=[test_output["recall"]]
    metric["f1_score"]=[test_output["f1_score"]]
    metric["roc_auc"]=[test_output["AUC"]]
    metric["pr_auc"]=[test_output["pr_auc"]]
    return metric

def metrics_df(output_dir, model_name):
    data_name=[x for x in os.listdir(output_dir) if x.split(".")[-1]=="csv"]
    data_name=sorted(data_name)
    df=pd.read_csv(os.path.join(output_dir , data_name[0]))
    metrics=metrics_read(df,model_name)
    for i in range(1,len(data_name)):
        df=pd.read_csv(os.path.join(output_dir , data_name[i]))
        metrics=pd.concat([metrics,metrics_read(df,model_name)],axis=0,ignore_index=True)
        
    metrics.drop_duplicates(subset=["recall"],inplace=True, keep="first")
    return metrics

In [ ]:
precision=[]
recall=[]

output_dir="/opt/omniai/work/instance1/jupyter/v3_new_email/Fine-Tuning/results/longformer_base_4096"
df=metrics_df(output_dir, "longformer_base")
precision.append(df["precision"].tolist())
recall.append(df["recall"].tolist())

output_dir="/opt/omniai/work/instance1/jupyter/v3_new_email/Fine-Tuning/results/longformer_base_4096_customized"
df=metrics_df(output_dir, "longformer_base")
precision.append(df["precision"].tolist())
recall.append(df["recall"].tolist())

output_dir="/opt/omniai/work/instance1/jupyter/v3_new_email/Fine-Tuning/results/longformer_large_4096"
df=metrics_df(output_dir, "longformer_large")
precision.append(df["precision"].tolist())
recall.append(df["recall"].tolist())

output_dir="/opt/omniai/work/instance1/jupyter/v3_new_email/Fine-Tuning/results/longformer_large_4096_customized"
df=metrics_df(output_dir, "longformer_large")
precision.append(df["precision"].tolist())
recall.append(df["recall"].tolist())

# output_dir="/opt/omniai/work/instance1/jupyter/v3_new_email/Fine-Tuning/results/bigbird_roberta_large"
# df=metrics_df(output_dir, "bigbird_large")
# precision.append(df["precision"].tolist())
# recall.append(df["recall"].tolist())

# output_dir="/opt/omniai/work/instance1/jupyter/v3_new_email/Fine-Tuning/results/bigbird_roberta_large_customized"
# df=metrics_df(output_dir, "bigbird_large")
# precision.append(df["precision"].tolist())
# recall.append(df["recall"].tolist())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Define precision and recall values for each model
# models = ['longformer-base','customized longformer-base', 'longformer-large','customized longformer-large', 'bigbird-large','customized bigbird-large']
models = ['longformer-base','customized longformer-base', 'longformer-large','customized longformer-large']

# markers = ['o', 's', 'D', 'x', '*',  'p']
# colors = ['blue', 'green', 'orange', 'red', 'purple','black']
markers = ['o', 's', 'D', 'x']
colors = ['blue', 'green', 'orange', 'red']

# Plot precision and recall
plt.figure(figsize=(8, 6))

# Iterate over models
for i in range(len(models)):
    plt.plot(recall[i], precision[i], marker=markers[i],  color=colors[i], label=models[i], linewidth=3, linestyle=":", markersize=8)

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve', fontsize=14)
plt.grid(True)

# Format axis values as percentages
ax = plt.gca()
ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=0))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=2))

# Set the legend
plt.legend()

# Show the plot
plt.show()